In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from google.colab import files

#***SMA***

In [ ]:
df1 = pd.read_csv('btc_6h.csv')
df1 = df1.set_index(pd.DatetimeIndex(df1['datetime'].values))

# plt.figure(figsize=(16,8))
# plt.title('close price history', fontsize = 18)
# plt.plot(df['close'])
# plt.xlabel('date' , fontsize = 18)
# plt.ylabel('close Price',fontsize = 18)
# plt.show()

def SMA(data, period = 60, column = 'close'):
    return data[column].rolling(window=period).mean()
df1['SMA20'] =SMA(df1, 200)
df1['SMA50'] =SMA(df1, 500)

df1['signal'] = np.where(df1['SMA20'] > df1['SMA50'], 1, 0)
df1['position'] = df1['signal'].diff()

df1['Buy'] = np.where(df1['position']== 1, df1['close'], np.NAN)
df1['sell'] = np.where(df1['position']== -1, df1['close'], np.NAN)

# plt.figure(figsize=(16,8))
# plt.title('close price history with buy & sell signal', fontsize = 18)
# plt.plot(df['close'], alpha = 0.5, label = 'close')
# plt.plot(df['SMA20'], alpha = 1, label = 'SMA20')
# plt.plot(df['SMA50'], alpha = 1, label = 'SMA50')
# plt.scatter(df.index, df['Buy'], alpha = 1, label = 'Buy Signal', marker = '^', color = 'green')
# plt.scatter(df.index, df['sell'], alpha = 1, label = 'sell Signal', marker = 'v', color = 'red')

# plt.xlabel('date' , fontsize = 18)
# plt.ylabel('close Price',fontsize = 18)
# plt.legend(['close', 'EMA Short', 'EMA long', 'Buy', 'sell'])
# plt.show()

df1['SMASig'] = 0
df1.loc[df1['position'] == 1, 'SMASig'] = 1
df1.loc[df1['position'] == -1, 'SMASig'] = -1
columns_to_drop = ['SMA20', 'SMA50', 'signal','position','Buy','sell']
df1 = df1.drop(columns=columns_to_drop)
df1.to_csv('(1).csv', index=False)

In [ ]:
df2 = pd.read_csv('(1).csv')
df2

,datetime,open,high,low,close,volume,SMASig
0,2018-01-01 05:30:00,13715.65,13715.65,13155.38,13558.99,2421.241182,0
1,2018-01-01 11:30:00,13539.00,13818.55,13001.13,13220.56,1876.934392,0
2,2018-01-01 17:30:00,13220.56,13330.00,12750.00,13022.00,2427.898747,0
3,2018-01-01 23:30:00,13007.87,13599.70,12940.00,13380.00,1883.841523,0
4,2018-01-02 05:30:00,13382.16,13850.00,12890.02,13166.95,3957.954564,0
...,...,...,...,...,...,...,...
5954,2022-01-30 05:30:00,38166.83,38300.00,37737.49,38139.39,4656.397760,0
5955,2022-01-30 11:30:00,38139.38,38359.26,37795.00,38110.02,6113.435610,0
5956,2022-01-30 17:30:00,38110.03,38153.82,37712.05,37991.00,4075.221880,0
5957,2022-01-30 23:30:00,37991.00,38230.00,37351.63,37881.76,6585.610020,0


# ***EMA***

In [ ]:
df2 = pd.read_csv('(1).csv')

df2 = df2.set_index(pd.DatetimeIndex(df2['datetime'].values))

df2['ema_short'] = df2['close'].ewm(span = 9, adjust = False).mean()
df2['ema_long'] = df2['close'].ewm(span = 21, adjust = False).mean()

df2['bullish'] = 0.0
df2['bullish'] = np.where(df2['ema_short'] > df2['ema_long'], 1, 0)

df2['crossover'] = df2['bullish'].diff()

# fig = plt.figure(figsize = (12,8))
# axl = fig.add_subplot(111, ylabel = 'price in $')

# df2['close'].plot(ax = axl, color = 'b', lw =3.)
# df2['ema_short'].plot(ax = axl, color = 'r', lw =3.)
# df2['ema_long'].plot(ax = axl, color = 'g', lw =3.)

# axl.plot(df2.loc[df2.crossover == 1].index,
#          df2.close[df2.crossover == 1],
#          '.', markersize = 5, color = 'g')

# axl.plot(df2.loc[df2.crossover == -1].index,
#          df2.close[df2.crossover == -1],
#          '.', markersize = 4, color = 'r')

# plt.legend(['close', 'EMA Short', 'EMA long', 'Buy', 'sell'])
# plt.title('EMA crossover')

df2['EMASig'] = 0
df2.loc[df2['crossover'] == 1, ['EMASig']] = 1
df2.loc[df2['crossover'] == -1, ['EMASig']] = -1
columns_to_drop = ['ema_short','ema_long','bullish','crossover']
df2 = df2.drop(columns=columns_to_drop)
df2.to_csv('(2).csv', index=False)

In [ ]:
df3 = pd.read_csv("(2).csv")
df3

,datetime,open,high,low,close,volume,SMASig,EMASig
0,2018-01-01 05:30:00,13715.65,13715.65,13155.38,13558.99,2421.241182,0,0
1,2018-01-01 11:30:00,13539.00,13818.55,13001.13,13220.56,1876.934392,0,0
2,2018-01-01 17:30:00,13220.56,13330.00,12750.00,13022.00,2427.898747,0,0
3,2018-01-01 23:30:00,13007.87,13599.70,12940.00,13380.00,1883.841523,0,0
4,2018-01-02 05:30:00,13382.16,13850.00,12890.02,13166.95,3957.954564,0,0
...,...,...,...,...,...,...,...,...
5954,2022-01-30 05:30:00,38166.83,38300.00,37737.49,38139.39,4656.397760,0,0
5955,2022-01-30 11:30:00,38139.38,38359.26,37795.00,38110.02,6113.435610,0,0
5956,2022-01-30 17:30:00,38110.03,38153.82,37712.05,37991.00,4075.221880,0,0
5957,2022-01-30 23:30:00,37991.00,38230.00,37351.63,37881.76,6585.610020,0,0


# ***MeanReversion***

In [ ]:
df3 = pd.read_csv("(2).csv")
# Assuming 'df' is your DataFrame with datetime as index and 'close' as closing prices
window_size = 20  # Choose an appropriate window size

# Calculate rolling mean and standard deviation
df3['rolling_mean'] = df3['close'].rolling(window=window_size).mean()
df3['rolling_std'] = df3['close'].rolling(window=window_size).std()

# Choose an appropriate threshold for mean deviation
threshold = 1.0  # Adjust as needed

# Generate signals
df3['buy_signal'] = df3['close'] < (df3['rolling_mean'] - threshold * df3['rolling_std'])
df3['sell_signal'] = df3['close'] > (df3['rolling_mean'] + threshold * df3['rolling_std'])

# # Plot actual prices, rolling mean, and buy/sell signals
# plt.figure(figsize=(10, 6))
# plt.plot(df['close'], label='Actual Prices', color='black')
# plt.plot(df['rolling_mean'], label='Rolling Mean', color='blue')
# plt.scatter(df.index[df['buy_signal']], df['close'][df['buy_signal']], label='Buy Signal', marker='^', color='green')
# plt.scatter(df.index[df['sell_signal']], df['close'][df['sell_signal']], label='Sell Signal', marker='v', color='red')
# plt.legend()
# plt.title('Actual Prices vs. Mean Reversion Strategy')
# plt.xlabel('Date')
# plt.ylabel('Price')
# plt.show()

df3['MeanRevSig'] = 0  # Initialize MeanRevSig column with zeros

# Apply conditions
df3.loc[(df3['buy_signal'] == True) & (df3['sell_signal'] == False), 'MeanRevSig'] = 1
df3.loc[(df3['buy_signal'] == False) & (df3['sell_signal'] == True), 'MeanRevSig'] = -1
columns_to_drop = ['rolling_mean','rolling_std','buy_signal','sell_signal']
df3 = df3.drop(columns=columns_to_drop)
df3.to_csv('(3).csv', index=False)

# ***Combining***

In [ ]:
df = pd.read_csv("(3).csv")

df['FinalSig'] = 0

# Define conditions using bitwise operators
condition_3 = (df['MeanRevSig'] + df['SMASig'] + df['EMASig'] == 3)
condition_2 = (df['MeanRevSig'] + df['SMASig'] + df['EMASig'] == 2)
condition_1 = (df['MeanRevSig'] + df['SMASig'] + df['EMASig'] == 1)
condition_minus_1 = (df['MeanRevSig'] + df['SMASig'] + df['EMASig'] == -1)
condition_minus_2 = (df['MeanRevSig'] + df['SMASig'] + df['EMASig'] == -2)
condition_minus_3 = (df['MeanRevSig'] + df['SMASig'] + df['EMASig'] == -3)

# Apply conditions to set FinalSig
df.loc[condition_3, 'FinalSig'] = 2
df.loc[condition_2, 'FinalSig'] = 2
df.loc[condition_1, 'FinalSig'] = 1
df.loc[condition_minus_1, 'FinalSig'] = -1
df.loc[condition_minus_2, 'FinalSig'] = -2
df.loc[condition_minus_3, 'FinalSig'] = -2


# ***Saving and Downloading***

In [ ]:
df.to_csv('btc_6hSignals.csv', index=False)

In [ ]:
files.download('btc_6hSignals.csv')